In [ ]:
using Pkg
Pkg.activate("/Users/jmaack/SIIP_latest") # working
# Pkg.activate("/Users/jmaack/SIIP") # working
Pkg.status()

In [ ]:
using StructJuMP
using ProgressiveHedging
const PH = ProgressiveHedging

using Ipopt

In [ ]:
nscen = 4
nbranch = 2
# Parameters
c = [10.0, 1.0]
d = 7.0
a = 16.0

α = 1.0
β = 1.0
γ = 1.0
δ = 1.0
ϵ = 1.0

s1 = 8.0
s2 = 4.0
s11 = 9.0
s12 = 16.0
s21 = 5.0
s22 = 18.0

# First stage
root_model = StructuredModel(num_scenarios=nbranch)
@variable(root_model, x[1:2] >= 0.0)
#@variable(root_model, F >= 0.0)
@objective(root_model, Min, sum(c.*x))
# @objective(root_model, Min, c*x + 10.0*F)
# @objective(root_model, Min, 0.5*F)
# @constraint(root_model, F >= 4.0)
# @constraint(root_model, x + F >= 5.0)
# @constraint(root_model, 2*x + 5 >= 1)

# Second stage
mid_model_1 = StructuredModel(parent=root_model, id=1, num_scenarios=nbranch)#, prob=0.5)
@variable(mid_model_1, y1 >= 0.0)
@objective(mid_model_1, Min, d*y1)
@constraint(mid_model_1, α*sum(x) + β*y1 == s1)
# @constraint(mid_model_1, α*x + β*y1 + 2.0*F == s1)
mid_model_2 = StructuredModel(parent=root_model, id=2, num_scenarios=nbranch)#, prob=0.5)
@variable(mid_model_2, y2 >= 0.0)
@objective(mid_model_2, Min, d*y2)
# @constraint(mid_model_2, y2 <= 10.0)
@constraint(mid_model_2, α*sum(x) + β*y2 == s2)
# @constraint(mid_model_2, α*x + β*y2 + 2.0*F == s2)
;

# # Third stage
# leaf_11 = StructuredModel(parent=mid_model_1, id=11, prob=0.75)
# # @variable(leaf_11, z11 >= 0.0)
# @variable(leaf_11, z11[1:2])
# @objective(leaf_11, Min, a*sum(z11[i]^2 for i in 1:2))
# @constraint(leaf_11, ϵ*x + γ*y1 + δ*sum(z11) + F == s11)

# leaf_12 = StructuredModel(parent=mid_model_1, id=12, prob=0.25)
# # @variable(leaf_12, z12 >= 0.0)
# @variable(leaf_12, z12[1:2])
# @objective(leaf_12, Min, a*sum(z12[i]^2 for i in 1:2))
# @constraint(leaf_12, ϵ*x + γ*y1 + δ*sum(z12) + F == s12)

# leaf_21 = StructuredModel(parent=mid_model_2, id=21, prob=0.75)
# # @variable(leaf_21, z21 >= 0.0)
# @variable(leaf_21, z21[1:2])
# @objective(leaf_21, Min, a*sum(z21[i]^2 for i in 1:2))
# @constraint(leaf_21, ϵ*x + γ*y2 + δ*sum(z21) + F == s21)

# leaf_22 = StructuredModel(parent=mid_model_2, id=22, prob=0.25)
# # @variable(leaf_22, z22 >= 0.0)
# @variable(leaf_22, z22[1:2])
# @objective(leaf_22, Min, a*sum(z22[i]^2 for i in 1:2))
# @constraint(leaf_22, ϵ*x + γ*y2 + δ*sum(z22) + F == s22)
;

# Progressive Hedging Solution

In [ ]:
@time (n, err, soln, phd) = PH.solve(root_model, with_optimizer(Ipopt.Optimizer, print_level=0, tol=1e-12), 
    25.0, atol=1e-8, max_iter=500)
println("Number of iterations: ", n)
println("L^2 error: ", err)
# print_values(phd)
println(soln)
# println(obj)

# Extensive Form Solution

In [ ]:
@time ef_model = PH.solve_extensive_form(root_model, with_optimizer(Ipopt.Optimizer, print_level=0))
for var in JuMP.all_variables(ef_model)
    println(var, " = ", JuMP.value(var))
end

In [ ]:
ef_model

# Something Seems Broken

In [ ]:
# using JuMP
# function print_values(ph_data::PH.PHData)
#     for (s,m) in ph_data.submodels
#         println("****** $s *******")
#         for v in JuMP.all_variables(m)
#             if !JuMP.is_fixed(v)
#                 println(v, " = ", JuMP.value(v))
#             else
#                 println(v, " = ", JuMP.fix_value(v))
#             end
#         end
#     end
#     return
# end

In [ ]:
nscen = 4
nbranch = 2
# Parameters
c = 2.0
d = 7.0

α = 1.0
β = 1.0

s1 = 8.0
s2 = 4.0

# First stage
root_model = StructuredModel(num_scenarios=nbranch)
@variable(root_model, x >= 0.0)
@variable(root_model, F >= 0.0)
@objective(root_model, Min, c*x + 10.0*F)

# Second stage
mid_model_1 = StructuredModel(parent=root_model, id=1, num_scenarios=nbranch)
@variable(mid_model_1, y1 >= 0.0)
@objective(mid_model_1, Min, d*y1)
@constraint(mid_model_1, α*x + β*y1 == s1)
mid_model_2 = StructuredModel(parent=root_model, id=2, num_scenarios=nbranch)
@variable(mid_model_2, y2 >= 0.0)
@objective(mid_model_2, Min, d*y2)
@constraint(mid_model_2, α*x + β*y2 == s2)
;

In [ ]:
(n, err, soln, phd) = PH.solve(root_model, with_optimizer(Ipopt.Optimizer, print_level=0, tol=1e-12), 
    25.0, atol=1e-8, max_iter=500)
println("Number of iterations: ", n)
println("L^2 error: ", err)
println(soln)

In [ ]:
@time ef_model = PH.solve_extensive_form(root_model, with_optimizer(Ipopt.Optimizer, print_level=0))
for var in JuMP.all_variables(ef_model)
    println(var, " = ", JuMP.value(var))
end